In [1]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
# from dashgrid.dgrid_components import PlotlyCandles as plc
from dashgrid import db_info as dbi

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate
import dash_table
import pandas as pd
import numpy as np
import json
import logging
import datetime
import functools
import random
import inspect
from pandasql import sqldf
import datetime,base64,io,pytz
import collections as cole

In [2]:
DEFAULT_LOG_PATH = './logfile.log'
DEFAULT_LOG_LEVEL = 'INFO'

def init_root_logger(logfile=DEFAULT_LOG_PATH,logging_level=DEFAULT_LOG_LEVEL):
    level = logging_level
    if level is None:
        level = logging.DEBUG
    # get root level logger
    logger = logging.getLogger()
    if len(logger.handlers)>0:
        return logger
    logger.setLevel(logging.getLevelName(level))

    fh = logging.FileHandler(logfile)
    fh.setLevel(logging.DEBUG)
    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger.addHandler(fh)
    logger.addHandler(ch)   
    return logger


In [3]:
logger = init_root_logger(logging_level='DEBUG')


In [4]:
def stop_callback(errmess,logger=None):
    m = "****************************** " + errmess + " ***************************************"     
    if logger is not None:
        logger.debug(m)
    raise PreventUpdate()


# Dlink: Link Dash components 

In [5]:
def slice_df(df_in,input_list):
    """!
        ### slice_df slices a DataFrame according to a list of values.
        1. The arg ```input_list``` contains values that coincide with columns of the DataFrame
        2. The method returns a new filtered DataFrame where:
         * the method filters each column using the respective value in ```input_list```
         * if there is a ```None``` in any item in the input list, the method ignores filtering on that column
        :param df_in: DataFrame to be sliced
        :param input_list: list of values to use as criteria for each column slice. 
                            A value of None will cause the method to not slice that column
        :return: new DataFrame that has been sliced
    """
    dfc = df_in.drop_duplicates().copy()
    cols = dfc.columns.values
    current_input = 0
    for i in range(len(input_list)):
        il = input_list[i]
        col = cols[i]
        if il is not None:
            try:                
                dfc = dfc[dfc[col]==il]
            except Exception as e:
                raise ValueError(e)
        current_input += 1
    return dfc


In [6]:
def choices_from_df(df_in,input_list,col_to_select):
    '''
    Create a list of dicts that conform to an options list for a Dash componet like RadioItems or Dropdown,where:
    1. Each dict in the list has a 'label' key, and a 'value' key
    2. ```df_in``` is sliced
    '''
    dfc = slice_df(df_in,input_list)
    if dfc is None or len(dfc)<1:
        return None,None
    unique_values = dfc[col_to_select].unique()
    choices = unique_values[0],[{'label':uv,'value':uv} for uv in unique_values]
    return choices



In [7]:
GRID_STYLE = {'display': 'grid',
              'border': '1px solid #000',
              'grid-gap': '8px 8px',
              'background-color':'#fffff9',
            'grid-template-columns': '1fr 1fr'}

def create_grid(
        component_array,num_columns=2,
        column_width_percents=None,
        additional_grid_properties_dict=None,
        wrap_in_loading_state=False,
        row_layout=None):
    gs = GRID_STYLE.copy()
    if row_layout is None:
        percents = [str(round(100/num_columns-.001,1))+'%' for _ in range(num_columns)] if column_width_percents is None else [str(c)+'%' for c in column_width_percents]
        perc_string = " ".join(percents)
        gs['grid-template-columns'] = perc_string 
    else:
        gs['grid-template-columns'] = row_layout
    if additional_grid_properties_dict is not None:
        for k in additional_grid_properties_dict.keys():
            gs[k] = additional_grid_properties_dict[k]           
    
    div_children = []
    for c in component_array:
        if type(c)==str:
            div_children.append(GridItem(c).html)
        elif hasattr(c,'html'):
            div_children.append(c.html)
        else:
            div_children.append(c)
    if wrap_in_loading_state:
        g = dcc.Loading(html.Div(div_children,style=gs),type='cube')
    else:
        g = html.Div(div_children,style=gs)
    return g

class GridItem():
    def __init__(self,child,html_id=None):
        self.child = child
        self.html_id = html_id
    @property
    def html(self):
        if self.html_id is not None:
            return html.Div(children=self.child,className='grid-item',id=self.html_id)
        else:
            return html.Div(children=self.child,className='grid-item')


In [8]:
#********** define useful css styles ***********************
borderline = 'none' #'solid'
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': borderline,
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
button_style_no_border={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}

blue_button_style={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#A9D0F5',#ed4e4e
    'vertical-align':'middle',
}

border_style={
    'line-height': '40px',
    'border':borderline + ' #000',
    'textAlign': 'center',
    'vertical-align':'middle',
}

table_like = {
    'display':'table',
    'width': '100%'
}

# define h4_like because h4 does not play well with dash_table
h4_like = { 
    'display': 'table-cell',
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-size' : '16px',
    'font-weight': 'bold',
    'width': '100%',
    'color':'#22aaff'
}


cte_title_style = { 
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-weight': 'bold',
    'color':'#22aaff'
}

DEFAULT_TIMEZONE = 'US/Eastern'


In [9]:
# ************************* define useful factory methods *****************

def parse_contents(contents):
    '''
    app.layout contains a dash_core_component object (dcc.Store(id='df_memory')), 
      that holds the last DataFrame that has been displayed. 
      This method turns the contents of that dash_core_component.Store object into
      a DataFrame.
      
    :param contents: the contents of dash_core_component.Store with id = 'df_memory'
    :returns pandas DataFrame of those contents
    '''
    c = contents.split(",")[1]
    c_decoded = base64.b64decode(c)
    c_sio = io.StringIO(c_decoded.decode('utf-8'))
    df = pd.read_csv(c_sio)
    # create a date column if there is not one, and there is a timestamp column instead
    cols = df.columns.values
    cols_lower = [c.lower() for c in cols] 
    if 'date' not in cols_lower and 'timestamp' in cols_lower:
        date_col_index = cols_lower.index('timestamp')
        # make date column
        def _extract_dt(t):
            y = int(t[0:4])
            mon = int(t[5:7])
            day = int(t[8:10])
            hour = int(t[11:13])
            minute = int(t[14:16])
            return datetime.datetime(y,mon,day,hour,minute,tzinfo=pytz.timezone(DEFAULT_TIMEZONE))
        # create date
        df['date'] = df.iloc[:,date_col_index].apply(_extract_dt)
    return df

def make_df(dict_df):
    if type(dict_df)==list:
        if type(dict_df[0])==list:
            dict_df = dict_df[0]
        return pd.DataFrame(dict_df,columns=dict_df[0].keys())
    else:
        return pd.DataFrame(dict_df,columns=dict_df.keys())

class BadColumnsException(Exception):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)



def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table',logger=None,
                  title_style=None):
    '''
    Create an instance of dash_table.DataTable, wrapped in an dash_html_components.Div
    
    :param dtable_id: The id for your DataTable
    :param df_in:     The pandas DataFrame that is the source of your DataTable (Default = None)
                        If None, then the DashTable will be created without any data, and await for its
                        data from a dash_html_components or dash_core_components instance.
    :param columns_to_display:    A list of column names which are in df_in.  (Default = None)
                                    If None, then the DashTable will display all columns in the DataFrame that
                                    it receives via df_in or via a callback.  However, the column
                                    order that is displayed can only be guaranteed using this parameter.
    :param editable_columns_in:    A list of column names that contain "modifiable" cells. ( Default = None)
    :param title:    The title of the DataFrame.  (Default = Dash Table)
    :param logger:
    :param title_style: The css style of the title. Default is dgrid_components.h4_like.
    '''
    # create logger 
    lg = init_root_logger() if logger is None else logger
    
    lg.debug(f'{dtable_id} entering create_dt_div')
    
    # create list that 
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        filter_action='none', # 'fe',
#         fixed_rows={'headers': True, 'data': 0},
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
#             'maxHeight':'450px','overflowX': 'scroll','overflowY':'scroll'
            'overflowY':'scroll'
        } ,
        
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            if any([c not in df.columns.values for c in columns_to_display]):
                m = f'{columns_to_display} are missing from input data. Your input Csv'
                raise BadColumnsException(m)           
            df = df[columns_to_display]
            
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    s = h4_like if title_style is None else title_style
    child_div = html.Div([html.Div(html.Div(title,style=s),style=table_like),dt])
    lg.debug(f'{dtable_id} exiting create_dt_div')
#     return child_div
    return dt


In [10]:
def create_xy_graph():
    pass

In [11]:
def recursive_grid_layout(app_component_list,current_component_index,gtcl,layout_components,wrap_in_loading_state=False):
    # loop through the gtcl, and assign components to grids
    for grid_template_columns in gtcl:
        if type(grid_template_columns)==list:
            layout_components.append(recursive_grid_layout(
                app_component_list,current_component_index, grid_template_columns, 
#                 layout_components,wrap_in_loading_state=True))
                layout_components,wrap_in_loading_state=False))
            continue
        # if this grid_template_columns item is NOT a list, process it normally
        sub_list_grid_components = []
        num_of_components_in_sublist = len(grid_template_columns.split(' '))
        for _ in range(num_of_components_in_sublist):
            # get the current component
            layout_ac = app_component_list[current_component_index]
            # add either the component, or it's html property to the sublist
            if hasattr(layout_ac, 'html'):
                layout_ac = layout_ac.html
            sub_list_grid_components.append(layout_ac)
            current_component_index +=1
        new_grid = create_grid(sub_list_grid_components, 
                        additional_grid_properties_dict={'grid-template-columns':grid_template_columns},
                        wrap_in_loading_state=wrap_in_loading_state)
        layout_components.append(new_grid)

def make_layout(comp_list,grid_template_columns_list):
    # for horizontal default
#     default_gtcl = [' '.join(['1fr' for _ in range(len(comp_list))])]
    # for vertical default
    default_gtcl = ['1fr' for _ in range(len(comp_list))]
    gtcl = default_gtcl if grid_template_columns_list is None else grid_template_columns_list
    
    # populate layout_components using recursive algo
    layout_list = []    
    recursive_grid_layout(comp_list,0,gtcl,layout_list)
    return    layout_list

In [12]:
class_converters = {
    dcc.Checklist:lambda v:v,
    dcc.DatePickerRange:lambda v:v,
    dcc.DatePickerSingle:lambda v:v,
    dcc.Dropdown:lambda v:v,
    dcc.Input:lambda v:v,
    dcc.Markdown:lambda v:v,
    dcc.RadioItems:lambda v:v,
    dcc.RangeSlider:lambda v:v,
    dcc.Slider:lambda v:v,
    dcc.Store:lambda v:v,
    dcc.Textarea:lambda v:v,
    dcc.Upload:lambda v:v,
}

html_members = [t[1] for t in inspect.getmembers(html)]
dcc_members = [t[1] for t in inspect.getmembers(dcc)]
all_members = html_members + dcc_members

class DashLink():
    def __init__(self,in_tuple_list, out_tuple_list,io_callback=None,
                 state_tuple_list= None,logger=None):
        self.logger = init_root_logger() if logger is None else logger
        _in_tl = [(k.id if type(k) in all_members else k,v) for k,v in in_tuple_list]
        _out_tl = [(k.id if type(k) in all_members else k,v) for k,v in out_tuple_list]
        self.output_table_names = _out_tl
        
        self.inputs = [Input(k,v) for k,v in _in_tl]
        self.outputs = [Output(k,v) for k,v in _out_tl]
        
        self.states = [] 
        if state_tuple_list is not None:
            _state_tl = [(k.id if type(k) in all_members else k,v) for k,v in state_tuple_list]
            self.states = [State(k,v) for k,v in _state_tl]
        
        self.io_callback = lambda input_list:input_list[0] 
        if io_callback is not None:
            self.io_callback = io_callback
                       
    def callback(self,theapp):
        @theapp.callback(
            self.outputs,
            self.inputs,
            self.states
            )
        def execute_callback(*inputs_and_states):
            l = list(inputs_and_states)
            if l is None or len(l)<1 or l[0] is None:
                stop_callback(f'execute_callback no data for {self.output_table_names}',self.logger)
            ret = self.io_callback(l)
            return ret if type(ret) is list else [ret]
        return execute_callback
        
def makeapp(comp_list,dlink_list,grid_layout_list=None):
    app = dash.Dash()
    layout_components = make_layout(comp_list,grid_layout_list)
    grid_html = html.Div(layout_components,style={'margin-left':'10px','margin-right':'10px'})
    app.layout=html.Div(grid_html)
    for dlink in dlink_list:
        dlink.callback(app)
    return app

In [13]:
# !jupyter nbconvert --to script easycomp.ipynb

In [14]:
# class SqlExpression():
#     """
#     Build and sql expresssion that can also be fed into a CTE.
#     Example:
    
#     """
#     def __init__(self,main_table_name,main_table_alias=None):
#         self.main_table_name = main_table_name
#         self.main_table_alias = main_table_name if main_table_alias is None else main_table_alias
#         self.join_expressions = []
#         self.where_clause_list = []
#         self.regular_col_list = []
#         self.aggregate_col_list = []
#         self.sort_order_list = []    
    
#     def get_main_table(self):
#         return self.main_table_name
    
#     def get_main_table_alias(self):
#         return self.main_table_alias

#     def add_join_expression(self,join_expression):
#         self.join_expressions.append(join_expression)

#     def add_where_clause(self,subject, verb,predicate):
#         wp = predicate.replace('%','%%') if (type(predicate) == str) else predicate
#         wv = f" {verb} "
#         ws = f"{subject} "
#         where_expression = ws + wv + wp
#         self.where_clause_list.append(where_expression)

#     def add_regular_display_cols(self,col_list):
#         self.regular_col_list.extend(col_list)
        
#     def add_aggregate_display_cols(self,agg_expression_list):
#         self.aggregate_col_list.extend(agg_expression_list)

#     def add_sort_cols(self,col_list):
#         self.sort_order_list.extend(col_list)
    
#     def make_expression(self):
#         join_str = '' if len(self.join_expressions)<1 else 'join ' + ' join '.join(self.join_expressions)
#         where_str = '' if len(self.where_clause_list)<1 else 'where ' + ' and '.join(self.where_clause_list)
#         reg_col_str = '*' if len(self.regular_col_list)<1 else ','.join(self.regular_col_list)
#         agg_col_str = ','.join(self.aggregate_col_list)
#         all_col_str = reg_col_str + ('' if len(agg_col_str)<1 else ',' + agg_col_str)
#         group_by_str = '' if len(agg_col_str)<1 else f'group by {reg_col_str}'
#         order_by_str = '' if len(self.sort_order_list) < 1 else 'order by ' + ','.join(self.sort_order_list)
#         expression = f"""
#         select {all_col_str} 
#         from {self.main_table_name} {self.main_table_alias}
#         {join_str}
#         {where_str}
#         {group_by_str}
#         {order_by_str}
#         """
#         return expression
    
# class CteBuilder():
#     def __init__(self,sql_expression_list,limit=20):
#         self.sql_expression_list = sql_expression_list
#         self.limit = limit
        
#     def make_sql(self):
#         sl = self.sql_expression_list
#         sle = [s.make_expression() for s in sl]
#         full_sql = 'with \n'
#         full_sql += '\n,'.join([f'f{i} as (' + sle[i] + ')' for i in range(len(sle))])
#         last_cte_name = str(len(sle)-1)
#         lm = '' if self.limit is None else f' limit {self.limit}'
#         full_sql += f"\nselect * from f{last_cte_name} {lm}"
#         return full_sql

In [15]:
class DashApp():
    def __init__(self):
        self.all_component_tuples = []
        self.all_dash_links = []
        
    def act_append(self,component_list,css_layout=None):
        cssl = css_layout
        if cssl is None:
            cssl = ' '.join(['1fr' for _ in component_list])
        component_already_exists = False
        for comp in component_list:
            for act in self.all_component_tuples:
                for existing_component in act[0]:
                    if comp.id==existing_component.id:
                        component_already_exists = True
                        break
        if not component_already_exists:
            new_tuple = (component_list,cssl)
            self.all_component_tuples.append(new_tuple)
        else:
            print(f'act_append component {comp.id} already in all_component_tuples')

    def make_component_and_css_lists(self):
        comp_list = []
        css_list = []
        for act in self.all_component_tuples:
            comp_list.extend(act[0])
            css_list.append(act[1])
        return comp_list,css_list

    def adl_append(self,dashlink):
        link_already_in_list = False
        for otn in dashlink.output_table_names:
            for adl in self.all_dash_links:
                for adl_otn in adl.output_table_names:
                    if otn == adl_otn:
                        link_already_in_list = True
                        break
        if not link_already_in_list:
            self.all_dash_links.append(dashlink)
        else:
            print(f'adl_append output {otn} already in output in all_dask_links')

            

In [16]:
dap = DashApp()

### Create components that allow you to use csv files as SQL Tables
___
##### title

##### tables upload and display
##### create sql statement
    * select main table
        * create joins
        * create where
        * create display cols
        * create agg cols
        * create order by cols
        * add sql statement

##### create cte
##### execute cte
___


___
### title
___

In [17]:
# set up top title 
title_markedown = """
# Sql Common Table Expression Builder
1. Load csv files to act as your SQL Tables
2. Use the panels below to enter criteria determining:
  * Your main table
  * Your joined tables
  * Your where clauses
  * Your non-aggregate columns to display
  * Your aggregate, if any, columns to display
3. Using the above, build a full SQL Expression
4. Using these SQL Expressions, build a full SQL Common Table Expression (CTE)
5. Click on the execute cte button to run the CTE query and return data
"""
mk1 = dcc.Markdown(title_markedown,id='mk1',style={'color':'black','textAlign': 'left',})
dap.act_append([mk1])


### tables upload and display

In [18]:
upload_header = html.Div(
    [html.H4("Choose csv files that represent the tables in your database:")],
    id='upload_header',
    style=blue_button_style)
dap.act_append([upload_header])
upload1 = dcc.Upload(
            id='upload1',
            children=html.Div(
                [html.A("Click to select csv files")],
                style={'borderStyle': 'dashed','textAlign': 'center'}),
            accept = '.csv',
            # Allow multiple files to be uploaded
            multiple=True)
dap.act_append([upload1])

dfcsv_init = pd.DataFrame({'table_name':[],'alias':[],'columns':[]})
# create the DashTable div that shows the data
dtcsv = create_dt_div('dtcsv',df_in=dfcsv_init,title=html.H4('Input Tables'),editable_columns_in=['alias'])

# make a DataFrame that holds information about each table you uploaded, 
#  like, name, alias, and names of columns
def make_dtcsv_table(input_list):
    file_names = input_list[0]
    file_names = [s.replace('.csv','') for s in file_names]
    aliases = [f't_{n}' for n in range(len(file_names))]
    
    # get contents columns so that you can display available columns to query with sql
    contents_list = input_list[1]    
    list_df = [parse_contents(content) for content in contents_list]
    col_lists = [', '.join(df.columns.values) for df in list_df]
    dft = pd.DataFrame({'table_name':file_names,'alias':aliases,'columns':col_lists})
    r = [dft.to_dict('records')]
    return r

# make link to update list of tables that are used in sql calls
dtcsv_link = DashLink([(upload1,'filename')],[('dtcsv','data')],make_dtcsv_table,state_tuple_list=[(upload1,'contents')])
dap.adl_append(dtcsv_link)

# create store to hold actual data in the csv files
tables_store = dcc.Store(id='tables_store')
def make_tables_store(input_list):
    # make dataframe dicts
    contents_list = input_list[0]    
    list_df = [parse_contents(content) for content in contents_list]
    
    file_names = input_list[1]
    file_names = [s.replace('.csv','') for s in file_names]
    dict_df = {file_names[i]:list_df[i].to_dict('records') for i in range(len(list_df)) }
    return [dict_df]
tables_store_link = DashLink([(upload1,'contents')],[(tables_store,'data')],make_tables_store,state_tuple_list=[(upload1,'filename')])
dap.adl_append(tables_store_link)

# build a div with the table and the store
dtcsv_cube = dcc.Loading(html.Div([dtcsv,tables_store]),type='cube',id='dtcsv_cube_loading')
dap.act_append([dtcsv_cube])


2020-03-04 00:33:46,115 - root - DEBUG - dtcsv entering create_dt_div
2020-03-04 00:33:46,122 - root - DEBUG - dtcsv exiting create_dt_div


### main table selection dropdowns

In [19]:
# select main table
main_dropdown = dcc.Dropdown(
    id='main_dropdown',placeholder="Select a Table",
    style={'textAlign': 'center'})
main_dropdown_div = html.Div(
    children=[html.H4('Select a main table from the dropdown:')],
    id='main_dropdown_div',
    style=blue_button_style)
dap.act_append([main_dropdown_div])
dap.act_append([main_dropdown])

def populate_main_dropdown_options(input_list):
    dict_df = input_list[0]
    if len(dict_df)<1:
        stop_callback('populate_main_dropdown_options no data',logger)
    df_tables = make_df(dict_df)
    table_names = df_tables.table_name.values
#     table_names = df_tables.alias.values
    ret = [{'label':tn,'value':tn} for tn in table_names]
    return [ret]
main_dropdown_link = DashLink(
    [('dtcsv','data')],[(main_dropdown,'options')],populate_main_dropdown_options)
dap.adl_append(main_dropdown_link)

main_table_store = dcc.Store(id='main_table_store',data=[])
dap.act_append([main_table_store])
def update_main_table_store(input_list):
    main_table_name = input_list[0]
    return [main_table_name]
main_table_store_link = DashLink([(main_dropdown,'value')],[(main_table_store,'data')],
                                update_main_table_store)
dap.adl_append(main_table_store_link)


### create left and right join table selections

In [20]:
# define header
join_header = html.Div([html.H4('Create SQL Join Statements')],
                style=blue_button_style,id='join_header')
dap.act_append([join_header])
# define dcc component, div and DashLink for selecting left table
join_left_radioitems = dcc.RadioItems(id='join_left_radioitems',labelStyle={"display":"block","vertical-align":"middle"})
join_left_radioitems_div = html.Div([html.Div('Select left join table:'),html.Div([join_left_radioitems])],id='join_left_radioitems_div')
def populate_join_left_radio_options(input_list):
    main_table_name = input_list[0]
    all_table_names = [d['value'] for d in input_list[1]]
#     join_table_names = [jt for jt in all_table_names if jt != main_table_name]
    join_table_names = all_table_names
    ret = [{'label':tn,'value':tn} for tn in join_table_names] #+ [{}]
    return [ret]
join_left_radioitems_link = DashLink(
    [(main_dropdown,'value')],[(join_left_radioitems,'options')],
    populate_join_left_radio_options,state_tuple_list=[(main_dropdown,'options')])
dap.adl_append(join_left_radioitems_link)

# define dcc component, div and DashLink for selecting right table
join_right_radioitems = dcc.RadioItems(id='join_right_radioitems',labelStyle={"display":"block","vertical-align":"middle"})
join_right_radioitems_div = html.Div([html.Div('Select right join table:'),html.Div([join_right_radioitems])],id='join_right_radioitems_div')
def populate_join_right_radio_options(input_list):
    main_table_name = input_list[0]
    all_table_names = [d['value'] for d in input_list[1]]
    join_table_names = [jt for jt in all_table_names if jt != main_table_name]
    ret = [{'label':tn,'value':tn} for tn in join_table_names] #+ [{}]
    return [ret]
join_right_radioitems_link = DashLink(
    [(join_left_radioitems,'value')],[(join_right_radioitems,'options')],
    populate_join_right_radio_options,state_tuple_list=[(join_left_radioitems,'options')])
dap.adl_append(join_right_radioitems_link)
dap.act_append([join_left_radioitems_div,join_right_radioitems_div])

### Define 3 sets of columns to use for left and right join

In [21]:
# define dcc component, div and DashLink for selecting left table columns
def populate_join_columns_dropdown_options(input_list):
    table_name = input_list[0]
    # this is the dataframe that holds the names, aliases and columns of the csv
    #   files that you uploaded
    dict_df = input_list[1]
    if len(dict_df)<1:
        stop_callback('populate_join_columns_dropdown_options no dataframe data',logger)
    df_tables = make_df(dict_df)
    df_tables_this_table = df_tables[df_tables.table_name==table_name]
    if (len(df_tables_this_table)<1):
        stop_callback(f'populate_join_columns_dropdown_options no data for table {table_name}',logger)
    columns = str(df_tables_this_table.iloc[0]['columns']).split(',')
    ret = [{'label':c,'value':c} for c in columns]
    return [ret]
class ColumnsDropdown():
    def __init__(self,id_base,join_left_radioitems,join_right_radioitems):
        # define left
        self.verb_options = [{'label':v,'value':v} for v in ['=','<','>','<=','>=','like','ilike']]
        self.join_left_columns_dropdown = dcc.Dropdown(id=f'join_left_columns_dropdown_{id_base}')
        self.join_left_columns_dropdown_div = html.Div([html.Div('Select left join column:'),html.Div([self.join_left_columns_dropdown])],
                                                       id=f'join_left_columns_dropdown_div_{id_base}')
        self.join_left_columns_dropdown_link = DashLink(
            [(join_left_radioitems,'value')],[(self.join_left_columns_dropdown,'options')],
            populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
        
        # define right
        self.join_right_columns_dropdown = dcc.Dropdown(id=f'join_right_columns_dropdown_{id_base}')
        self.join_right_columns_dropdown_div = html.Div([html.Div('Select right join column:'),html.Div([self.join_right_columns_dropdown])],
                                                       id=f'join_right_columns_dropdown_div_{id_base}')
        self.join_right_columns_dropdown_link = DashLink(
            [(join_right_radioitems,'value')],[(self.join_right_columns_dropdown,'options')],
            populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
        
        # define dcc component, div and DashLink for selecting join verb
        self.join_verb_dropdown = dcc.Dropdown(id=f'join_verb_dropdown_{id_base}',value='=',options=self.verb_options,placeholder="select verb")
        self.join_verb_dropdown_div = html.Div([html.Div(['Select a join verb:']),html.Div(self.join_verb_dropdown)],id=f'join_verb_dropdown_div_{id_base}')
#         self.div = html.Div(children=[
#             self.join_left_columns_dropdown_div,
#             self.join_verb_dropdown_div,
#             self.join_right_columns_dropdown_div])
        self.div = html.Div(children=[
            self.join_left_columns_dropdown,
            self.join_verb_dropdown,
#             self.join_right_columns_dropdown],style={'display':'grid','grid-template-columns':'1fr 1fr 1fr'})
            self.join_right_columns_dropdown])
        self.links = [self.join_left_columns_dropdown_link,
                     self.join_right_columns_dropdown_link]

# create a button that create the join sql from the dropdowns
cd1 = ColumnsDropdown('cd1',join_left_radioitems,join_right_radioitems)
cd2 = ColumnsDropdown('cd2',join_left_radioitems,join_right_radioitems)
cd3 = ColumnsDropdown('cd3',join_left_radioitems,join_right_radioitems)

for cd in [cd1,cd2,cd3]:
    for lnk in cd.links:
        dap.adl_append(lnk)
#         div_list = [
#             cd.join_left_columns_dropdown_div,
#             cd.join_verb_dropdown_div,
#             cd.join_right_columns_dropdown_div]
#         dap.act_append(div_list)

join_add_button = html.Button('Add join', id='join_add_button')
cd_html = html.Div([cd1.div,cd2.div,cd3.div,join_add_button],id="cd_html",
                   style={'display':'grid','grid-template-columns':'30% 30% 30% 10%'})
dap.act_append([cd_html])
# dap.act_append([join_add_button])


### create the div tha holds the join statements, and get's updated via the join_add_button

In [22]:

join_clauses_text = dcc.Textarea(id='join_clauses_text',rows=5)
dap.act_append([join_clauses_text])

join_store = dcc.Store(id='join_store',data=[])
dap.act_append([join_store])

def update_join_store(input_list):
    # Step 1: get an previous join statements in join_list
    join_list = input_list[1]
    
    # Step 2: get the left and right table names
    left_table = input_list[2]
    if any([v is None for v in [left_table,join_list]]):
        stop_callback('update_join_store join info missing',logger)
    right_table = input_list[3]
    
    # Step 3: get the left and right columns, and the verbs that link them
    left_columns = [input_list[4+i*3] for i in range(3)]
    verbs = [input_list[5+i*3] for i in range(3)]
    right_columns = [input_list[6+i*3] for i in range(3)]
    
    # Step 4: combine all of the above into a "multi-on" join expression
    join_text = ''
    if input_list[4] is  None:
        stop_callback('update_join_store no column info specified',logger)
    join_text = f"join {left_table} on "
    for i in range(3): 
        if input_list[4+i*3] is not None:
#             next_join_clause = f"{left_table}.{input_list[4+i*3]} {input_list[5+i*3]} {right_table}.{input_list[6+i*3]}"
            next_join_clause = f"{left_table}.{left_columns[i]} {verbs[i]} {right_table}.{right_columns[i]}"
            if i>0:
                next_join_clause = ' and ' + next_join_clause
            join_text += next_join_clause
                    
    # Step 5: append the new join to the join_list, and send the list back to the join_store
    join_list.append(join_text)
    return [join_list]

join_store_link = DashLink([(join_add_button,'n_clicks')],[(join_store,'data')],
                          update_join_store,
                          state_tuple_list=[
                              (join_store,'data'),
                              (join_left_radioitems,'value'),
                              (join_right_radioitems,'value'),
                              (cd1.join_left_columns_dropdown,'value'),
                              (cd1.join_verb_dropdown,'value'),
                              (cd1.join_right_columns_dropdown,'value'),
                              (cd2.join_left_columns_dropdown,'value'),
                              (cd2.join_verb_dropdown,'value'),
                              (cd2.join_right_columns_dropdown,'value'),
                              (cd3.join_left_columns_dropdown,'value'),
                              (cd3.join_verb_dropdown,'value'),
                              (cd3.join_right_columns_dropdown,'value')
                          ])
dap.adl_append(join_store_link)

def update_join_clauses_text(input_list):
    join_statements = input_list[0] 
    join_text = '\n'.join(join_statements)
    return [join_text]
sqldiv_update_link = DashLink([(join_store,'data')],[(join_clauses_text,'value')],update_join_clauses_text)
dap.adl_append(sqldiv_update_link)


### create columns to display

In [23]:
display_cols_header = html.Div([html.H4('Select columns to display')],
                style=blue_button_style,id='display_cols_header')
dap.act_append([display_cols_header])

# Add these components to DashApp, so that they are all on one row
display_cols_table = dcc.Dropdown(id='display_cols_table')
display_cols_colummns = dcc.Checklist(id='display_cols_colummns')
display_cols_expression = dcc.Input(id='display_cols_expression')
display_cols_add = html.Button('Add Display Columns',id='display_cols_add')
dap.act_append([display_cols_table,display_cols_colummns,display_cols_expression,
               display_cols_add])

# Add the TextArea that displays the sql clause for the display columns to DashApp
display_cols_text = dcc.Textarea(id='display_cols_text',rows = 4)
dap.act_append([display_cols_text])

# add the store, which you will not see even though it occupies a row
display_cols_store = dcc.Store(id='display_cols_store',data=[])
dap.act_append([display_cols_store])

# define the DashLink methods used in the DashLinks below
def build_display_cols_table_options(input_list):
    return [input_list[0]]

def build_display_cols_store(input_list):
    prev_cols = [] if input_list[1] is None else input_list[1] 
    current_table = input_list[2]
    current_columns = input_list[3]
    print(f"build_display_cols_store current_columns {current_columns}")
    current_expression = input_list[4]
    if current_table is None or current_columns is None:
        stop_callback('build_display_cols_store no table or columns selected',logger)
    cols = [f"{current_table}.{c}" for c in current_columns]
    if len(cols)<1 and current_expression is None:
        stop_callback('build_display_cols_store no table or columns selected',logger) 
    ce_list = [] if current_expression is None else [current_expression]
    ret = prev_cols + cols + ce_list
    ret = list(cole.OrderedDict.fromkeys(ret).keys())
    return [ret]    

def build_display_cols_text(input_list):
    return [','.join(input_list[0])]

# Create the DashLinks
# DashLink to update the tables displayed in display_cols_table
display_cols_table_lefttable_link = DashLink(
    [(join_left_radioitems,'options')],
    [(display_cols_table,'options')],build_display_cols_table_options)
# DashLink to update the columns for that table that are displayed
display_cols_table_columns_link = DashLink(
    [(display_cols_table,'value')],
    [(display_cols_colummns,'options')],
    populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
# DashLink that clears column checks when a new table is selected
display_cols_cols_reset_link = DashLink(
    [(display_cols_table,'value')],
    [(display_cols_colummns,'value')],
    lambda input_list: [''])
# DashLink to update dcc.Store that holds all column choices held 
#   throught all clicks of display_cols_add
display_cols_store_link = DashLink(
    [(display_cols_add,'n_clicks')],
    [(display_cols_store,'data')],
    build_display_cols_store,
    state_tuple_list=[
        (display_cols_store,'data'),
        (display_cols_table,'value'),
        (display_cols_colummns,'value'),
        (display_cols_expression,'value')
    ])
# DashLink to update TextArea that shows column choices
display_cols_store_text_link = DashLink(
    [(display_cols_store,'data')],[(display_cols_text,'value')],
    build_display_cols_text)

# add all DashLinks to the DashApp
for adl in [display_cols_table_lefttable_link,
            display_cols_table_columns_link,
            display_cols_cols_reset_link,
            display_cols_store_link,
            display_cols_store_text_link]:
    dap.adl_append(adl)
    

### create where clause builders

In [24]:
where_header = html.Div([html.H4('Create where expressions')],
                style=blue_button_style,id='where_header')
dap.act_append([where_header])
where_subject_table = dcc.Dropdown(id='where_subject_table',placeholder='Select a Table:')
where_subject_column = dcc.Dropdown(id='where_subject_column',placeholder='Select a Column:')
verb_options = [{'label':v,'value':v} for v in ['=','<','>','<=','>=','like','ilike']]
where_verb = dcc.Dropdown(
    id='where_verb',value='=',
    options=verb_options,placeholder="select verb")
where_object_table = dcc.Dropdown(id='where_object_table')
where_object_column = dcc.Dropdown(id='where_object_column')
where_object_value = dcc.Input(id='where_object_value')
where_add_clause = html.Button('Add Where Clause',id='where_add_clause')
where_store = dcc.Store(id='where_store',data=[])
wcl = [where_subject_table,where_subject_column,where_verb,
      where_object_table,where_object_column,where_object_value,
      where_add_clause]
dap.act_append(wcl)
dap.act_append([where_store])

where_text = dcc.Textarea(id='where_text',rows=4)
dap.act_append([where_text])

# def build_where_table_lefttable(input_list):
#     return [input_list[0]]
def build_where_add_clause_text(input_list):
    if input_list is None or len(input_list)<1 or input_list[0] is None:
        stop_callback('build_where_add_clause_text no data',logger)
        pass
def build_where_store(input_list):
    prev_where_clauses = [] if input_list[1] is None else input_list[1] 
    current_subject_table = input_list[2]
    current_verb = input_list[3]
    current_subject_column = input_list[4]
    current_object_table = input_list[5]
    current_object_column = input_list[6]
    current_object_value = input_list[7]
    
    if current_verb is None:
        stop_callback('build_where_store no verb selected',logger)
    if current_subject_column is None:
        stop_callback('build_where_store no subject column selected',logger)
    if current_object_column is None and current_object_value is None:
        stop_callback('build_where_store no object information selected',logger)
    subj_col = f"{current_subject_table}.{current_subject_column}"
    # for object, use either current_object_table and current_object_column or
    #    current_object_input
    obj_col = f"{current_object_table}.{current_object_column}" 
    if current_object_value is not None:
        obj_col = current_object_value
    
    ret = prev_where_clauses + [f"{subj_col} {current_verb} {obj_col}"]
    ret = list(cole.OrderedDict.fromkeys(ret).keys())
    return [ret]    

def build_where_text(input_value):
    where_clause_list = input_value[1]
    if where_clause_list is None or len(where_clause_list)<1:
         stop_callback('build_where_text no where clauses created',logger)
    ret = ' and '.join(where_clause_list)
    return [ret]

where_subject_table_lefttable_link = DashLink(
    [(join_left_radioitems,'options')],
    [(where_subject_table,'options')],
    lambda input_list:[input_list[0]])
where_subject_table_column_link =  DashLink(
    [(where_subject_table,'value')],
    [(where_subject_column,'options')],
    populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
where_object_table_lefttable_link = DashLink(
    [(join_left_radioitems,'options')],
    [(where_object_table,'options')],
    lambda input_list:[input_list[0]])
where_object_table_column_link = DashLink(
    [(where_object_table,'value')],
    [(where_object_column,'options')],
    populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
where_store_link = DashLink(
    [(where_add_clause,'n_clicks')],
    [(where_store,'data')],
    build_where_store,
    state_tuple_list=[
        (where_store,'data'),
        (where_subject_table,'value'),
        (where_verb,'value'),
        (where_subject_column,'value'),
        (where_object_table,'value'),
        (where_object_column,'value'),
        (where_object_value,'value')
    ])
where_add_clause_link = DashLink(
    [(where_add_clause,'n_clicks')],
    [(where_text,'value')],
    build_where_text,
    state_tuple_list=[(where_store,'data')])

for wclink in [where_subject_table_lefttable_link,where_subject_table_column_link,
              where_object_table_lefttable_link,where_object_table_column_link,
              where_store_link,where_add_clause_link]:
    dap.adl_append(wclink)

In [25]:
# agg_cols
agg_cols_header = html.Div([html.H4('Create aggregate function columns to display')],
                style=blue_button_style,id='agg_cols_header')
dap.act_append([agg_cols_header])
agg_cols_function_options = [{'label':v,'value':v} for v in ['min','max','sum','avg','std','count']]
agg_cols_function = dcc.Dropdown(
    id='agg_cols_function',value='=',
    options=agg_cols_function_options,placeholder="select an aggregate function")
agg_cols_table = dcc.Dropdown(id='agg_cols_table',placeholder='Select a Table:')
agg_cols_column = dcc.Dropdown(id='agg_cols_column',placeholder='Select a Column')
agg_cols_freeform = dcc.Input(id='agg_cols_freeform',placeholder='Enter an agg expression')
agg_cols_alias = dcc.Input(id='agg_cols_alias',placeholder='Enter an alias for this agg column')

agg_cols_add = html.Button('Add Aggregte Columns',id='agg_cols_add')
agl = [agg_cols_function,agg_cols_table,agg_cols_column,
      agg_cols_freeform,agg_cols_alias,agg_cols_add]
dap.act_append(agl)
agg_cols_store = dcc.Store(id='agg_cols_store',data=[])
dap.act_append([agg_cols_store])

agg_cols_text = dcc.Textarea(id='agg_cols_text',rows=4)
dap.act_append([agg_cols_text])

def build_agg_cols_add_store_link(input_list):
    prev_agg_cols = input_list[1]
    agg_cols_function_option = input_list[2]
    agg_cols_table = input_list[3]
    agg_cols_column = input_list[4]
    agg_cols_alias = '' if input_list[5] is None else input_list[5]
    agg_cols_freeform = input_list[6]
    if  (agg_cols_function_option is None and agg_cols_column is None) and agg_cols_freeform is None:
         stop_callback('build_agg_cols_add_store_link no inputs',logger)
    ret = agg_cols_freeform
    if agg_cols_freeform is None or len(agg_cols_freeform.strip())<1:
        # use agg/table/column
        ret = (agg_cols_function_option,f"{agg_cols_table}.{agg_cols_column}",agg_cols_alias)
    return [prev_agg_cols + [ret]]
def build_agg_cols_store_text_link(input_list):
    agg_cols_store = input_list[0]
    agg_col_aliases = [a[2] for a in agg_cols_store]
    agg_col_aliases = ['' if (a is None or len(a.strip())<1) else f' as {a}' for a in agg_col_aliases]
    agg_cols = [f"{agg_col_tuple[0]}({agg_col_tuple[1]})" for agg_col_tuple in agg_cols_store]
    agg_cols = [agg_cols[i] + agg_col_aliases[i] for i in range(len(agg_cols))]
    return ','.join(agg_cols)

agg_cols_table_lefttable_link = DashLink(
    [(join_left_radioitems,'options')],
    [(agg_cols_table,'options')],
    lambda input_list:[input_list[0]])
agg_cols_table_column_link =  DashLink(
    [(agg_cols_table,'value')],
    [(agg_cols_column,'options')],
    populate_join_columns_dropdown_options,state_tuple_list=[('dtcsv','data')])
agg_cols_add_store_link = DashLink(
    [(agg_cols_add,'n_clicks')],[(agg_cols_store,'data')],
    build_agg_cols_add_store_link,
    state_tuple_list=[
        (agg_cols_store,'data'),
        (agg_cols_function,'value'),
        (agg_cols_table,'value'),
        (agg_cols_column,'value'),
        (agg_cols_alias,'value'),
        (agg_cols_freeform,'value')
    ])
agg_cols_store_text_link = DashLink(
    [(agg_cols_store,'data')],[(agg_cols_text,'value')],
    build_agg_cols_store_text_link)

for l in [agg_cols_table_lefttable_link,agg_cols_table_column_link,
         agg_cols_add_store_link,agg_cols_store_text_link]:
    dap.adl_append(l)


### Create sql statement

In [26]:
sql_build_div = html.Div([html.H4("Build SQL Expression From all entries above")],
                        id='sql_build_div',style=blue_button_style)
dap.act_append([sql_build_div])
sql_build_button = html.Button('Build SQL Expression:',id='sql_build_button',style={'textAlign':'center'})
dap.act_append([sql_build_button])
sql_expression_text = dcc.Textarea(id='sql_expression_text',rows=10)
dap.act_append([sql_expression_text])


def build_sql_expression_text(input_list):
    main_dropdown = input_list[1]
    if main_dropdown is None or len(main_dropdown)<1:
        stop_callback('build_sql_expression_text no main table selected',logger)        
    join_clauses_text = '' if input_list[2] is None else input_list[2]
    display_cols_text = input_list[3]
    where_text = '' if input_list[4] is None else input_list[4]
    if len(where_text.strip())>0:
        where_text = 'where ' + where_text
    agg_cols_text = '' if input_list[5] is None else input_list[5]
    agg_cols_store = input_list[6]
    
    agg_cols_to_add = ''
    groupby = ''
    if len(agg_cols_text.strip())>0:
        groupby_cols = [agg_col[1] for agg_col in agg_cols_store]
        groupby_cols_text = ','.join(groupby_cols)
        groupby = f"group by {groupby_cols_text}"
        agg_cols_to_add = ',' + agg_cols_text
    
    sql = f"""
    select {display_cols_text}{agg_cols_to_add}
    from {main_dropdown}
    {join_clauses_text}
    {where_text}
    {groupby}
    limit 1000
    """
    return [sql]

sql_expression_button_text_link = DashLink(
    [(sql_build_button,'n_clicks')],[(sql_expression_text,'value')],
    build_sql_expression_text,
    state_tuple_list=[
        (main_dropdown,'value'),
        (join_clauses_text,'value'),
        (display_cols_text,'value'),
        (where_text,'value'),
        (agg_cols_text,'value'),
        (agg_cols_store,'data')
    ])
dap.adl_append(sql_expression_button_text_link)


### Define sql_run_button, and div that shows sql results

In [27]:
# define running sql
sql_run_button = html.Button('Execute SQL:',id='sql_run_button')
dap.act_append([sql_run_button])
sqldiv = html.Div(children=[],id='sqldiv')
sqlloading = dcc.Loading(sqldiv,type='cube',id='sqlloading')
dap.act_append([sqlloading])

def make_sqldiv(input_list):
    if input_list is None or len(input_list)<3 or input_list[1] is None:
        stop_callback('make_sqldiv no data',logger)
    sql = input_list[1]
    dict_df = input_list[2]
    if dict_df is None:
        stop_callback('make_sqldiv no csv data selected',logger)
    for k in dict_df.keys():
        globals()[k] = make_df(dict_df[k])
    df_sql_results = sqldf(sql,globals())
    dt_sql_results = create_dt_div('dtsql',df_in=df_sql_results,title=html.H4('Input Tables'))
    return [dt_sql_results]
sqlin_link = DashLink([(sql_run_button,'n_clicks')],[(sqldiv,'children')],make_sqldiv,state_tuple_list=[(sql_expression_text,'value'),(tables_store,'data')])
dap.adl_append(sqlin_link)


In [28]:
# create and run app
components,gtcl = dap.make_component_and_css_lists()
app = makeapp(components,dap.all_dash_links,gtcl)
app.run_server(host='127.0.0.1',port=8500)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-03-04 00:33:46,395 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-03-04 00:33:48,550 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:48] "GET / HTTP/1.1" 200 -
2020-03-04 00:33:48,640 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:48] "GET /assets/custom.css?m=1583299996.0 HTTP/1.1" 200 -
2020-03-04 00:33:48,653 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:48] "GET /assets/styles.css?m=1583299960.0 HTTP/1.1" 200 -
2020-03-04 00:33:48,753 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:48] "GET /_dash-layout HTTP/1.1" 200 -
2020-03-04 00:33:48,754 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:48] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-03-04 00:33:48,973 - root - DEBUG - ****************************** execute_callback no data for [('agg_cols_table', 'options')] ***************************************
2020-03-04 00:33:48,975 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:48] "POST /_dash-update-componen

2020-03-04 00:33:49,456 - root - DEBUG - ****************************** execute_callback no data for [('display_cols_colummns', 'options')] ***************************************
2020-03-04 00:33:49,458 - root - DEBUG - ****************************** execute_callback no data for [('agg_cols_column', 'options')] ***************************************
2020-03-04 00:33:49,459 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:49] "POST /_dash-update-component HTTP/1.1" 204 -
2020-03-04 00:33:49,460 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:49] "POST /_dash-update-component HTTP/1.1" 204 -
2020-03-04 00:33:49,461 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:49] "POST /_dash-update-component HTTP/1.1" 204 -
2020-03-04 00:33:49,470 - root - DEBUG - ****************************** execute_callback no data for [('where_object_column', 'options')] ***************************************
2020-03-04 00:33:49,472 - werkzeug - INFO - 127.0.0.1 - - [04/Mar/2020 00:33:49] "POST 

In [29]:
# # test checkbox behavior
# rops = [{'label':v,'value':v} for v in 'abcde']
# radio_button = dcc.RadioItems(id='radio_button',options=rops)
# check_list = dcc.Checklist(id='check_list')
# def build_check_list(input_list):
#     choices = [f'c{i}' for i in range(4)]
#     options = [{'label':v,'value':v} for v in choices] 
#     return [options]
# check_list_link = DashLink(
#     [(radio_button,'value')],[(check_list,'options')],
#                            build_check_list) 
# def build_check_list_reset(input_list):
#     return ['']
# check_list_reset = DashLink(
#     [(radio_button,'value')],[(check_list,'value')],build_check_list_reset)
# test_dap = DashApp()
# test_dap.adl_append(check_list_link)
# test_dap.adl_append(check_list_reset)
# test_dap.act_append([radio_button,check_list])
# testcomps,testgtcl = test_dap.make_component_and_css_lists()
# app = makeapp(testcomps,test_dap.all_dash_links,testgtcl)
# app.run_server(host='127.0.0.1',port=8500)


In [30]:
[c for c in dir(DashApp) if c[0] != '_']

['act_append', 'adl_append', 'make_component_and_css_lists']

# End